In [1]:
import pandas as pd
import numpy as np

In [2]:
# cross walk and filter top 15 CBSA

top15_cities = {
    'New York-Newark-Jersey City, NY-NJ-PA': 35620, 
    'Los Angeles-Long Beach-Anaheim, CA': 31080,
    'Chicago-Naperville-Elgin, IL-IN-WI': 16980,
    'Dallas-Fort Worth-Arlington, TX': 19100,
    'Houston-The Woodlands-Sugar Land, TX': 26420,
    'Washington-Arlington-Alexandria, DC-VA-MD-WV': 47900,
    'Miami-Fort Lauderdale-Pompano Beach, FL': 33100,
    'Philadelphia-Camden-Wilmington, PA-NJ-DE-MD': 37980,
    'Atlanta-Sandy Springs-Alpharetta, GA': 12060,
    'Phoenix-Mesa-Chandler, AZ': 38060,
    'Boston-Cambridge-Newton, MA-NH': 14460,
    'San Francisco-Oakland-Berkeley, CA': 41860,
    'Riverside-San Bernardino-Ontario, CA': 40140,
    'Detroit-Warren-Dearborn, MI': 19820,
    'Seattle-Tacoma-Bellevue, WA': 42660
    }

cross_walk = pd.read_csv('./Usage/us_xwalk.csv.gz', compression='gzip')[['cbsa', 'zcta']].drop_duplicates()
top15_crosswalk = cross_walk.set_index('cbsa').loc[top15_cities.values()].reset_index()['zcta']
top15_crosswalk

0        8817
1        7060
2        8733
3        7712
4        8501
        ...  
4726    98158
4727    98050
4728    98164
4729    98154
4730    98174
Name: zcta, Length: 4731, dtype: int64

In [3]:
# dist to CBD: log(dist_to_cbd)
cbd_info = pd.read_csv('./Usage/zori_panel_zips.csv')[['zip', 'dist_to_cbd']]
cbd_info['dist_to_cbd'] = np.log(cbd_info.dist_to_cbd)
cbd_info

,zip,dist_to_cbd
0,1001,8.587381
1,1002,10.344846
2,1085,9.660391
3,1420,10.484953
4,1453,10.263926
...,...,...
5529,99507,9.284990
5530,99508,8.405252
5531,99517,8.231623
5532,99577,10.454324


In [4]:
internet_link = './Output/final/INTERET_ZCTA.csv.gz'
internet_amenities = pd.read_csv(internet_link, compression='gzip')
internet_amenities

,GEO_ID,Computer,WirelessDevice,Cellular,CableFiber,SatelliteInternet,year
0,601.0,0.380,0.030,0.246,0.167,0.042,2017.0
1,602.0,0.436,0.029,0.248,0.329,0.031,2017.0
2,603.0,0.466,0.023,0.330,0.419,0.118,2017.0
3,606.0,0.330,0.028,0.081,0.213,0.013,2017.0
4,610.0,0.490,0.028,0.295,0.326,0.017,2017.0
...,...,...,...,...,...,...,...
200023,99923.0,0.640,0.520,0.640,0.000,0.000,2022.0
200024,99925.0,0.594,0.073,0.409,0.187,0.324,2022.0
200025,99926.0,0.624,0.087,0.740,0.573,0.076,2022.0
200026,99927.0,1.000,0.695,1.000,0.000,1.000,2022.0


In [5]:
ocupation_link = './Output/final/OCCUPATION_ZCTA.csv.gz'
occupation_score = pd.read_csv(ocupation_link, compression='gzip')
occupation_score.head()

,GEO_ID,REMOTE,REMOTE_PPL,year
0,1001,0.390959,706,2016
1,1002,0.537808,399,2016
2,1003,0.333762,5,2016
3,1005,0.380188,160,2016
4,1007,0.426538,540,2016


In [6]:
#merge all internet, occupation, cdb_info into 1 dataframe
all_iv = pd.merge(left = occupation_score, right = internet_amenities,
                  how = 'inner',
                  on = ['GEO_ID', 'year'])
all_iv = pd.merge(left = all_iv, right = cbd_info, how = 'inner',
                  left_on = 'GEO_ID', right_on='zip')

all_iv_report = pd.merge(left=all_iv, right=top15_crosswalk, how = 'inner',
                  left_on='GEO_ID', right_on='zcta')

all_iv = all_iv.drop(columns=['zip']).drop_duplicates()

all_iv.head()


,GEO_ID,REMOTE,REMOTE_PPL,year,Computer,WirelessDevice,Cellular,CableFiber,SatelliteInternet,dist_to_cbd
0,1001,0.404229,699,2017,0.777,0.043,0.409,0.722,0.030,8.587381
1,1001,0.397692,572,2018,0.799,0.041,0.485,0.742,0.027,8.587381
2,1001,0.399354,599,2019,0.801,0.042,0.581,0.689,0.025,8.587381
3,1001,0.387580,548,2020,0.785,0.043,0.673,0.695,0.021,8.587381
4,1001,0.383150,581,2021,0.784,0.055,0.726,0.679,0.022,8.587381


In [7]:
all_iv_report = all_iv_report.describe()[[col for col in all_iv.columns if col not in ('year', 'GEO_ID', 'zcta')]]
all_iv_report.iloc[:, :4].to_latex('./Output/SumStat/all_iv_1.tex')
all_iv_report.iloc[:, 4:].to_latex('./Output/SumStat/all_iv_2.tex')
all_iv_report

,REMOTE,REMOTE_PPL,Computer,WirelessDevice,Cellular,CableFiber,SatelliteInternet,dist_to_cbd
count,14220.000000,14220.000000,14220.000000,14220.000000,14220.000000,14220.000000,14220.000000,14220.000000
mean,0.404380,1542.653305,0.826442,0.034965,0.727234,0.759293,0.056164,10.027997
std,0.070458,1115.826110,0.105143,0.013791,0.146599,0.113845,0.033022,0.939797
min,0.239120,12.000000,0.314000,0.014000,0.179000,0.272000,0.000000,4.848175
25%,0.351771,795.000000,0.773000,0.027000,0.628000,0.698000,0.033000,9.553891
50%,0.395774,1303.000000,0.850000,0.032000,0.754000,0.783000,0.049000,10.166492
75%,0.449614,1987.000000,0.905000,0.039000,0.848000,0.845000,0.072000,10.677410
max,0.667122,10777.000000,1.000000,0.408000,1.000000,1.000000,0.330000,12.067979


Total Population Only

In [8]:
migration_link = './Output/final/MIGRATION_TOTAL_TOP15.csv.gz'
DV_population_top15 = pd.read_csv(migration_link, compression='gzip')
groups = [
    'PPL1Y', 
    'AGE',
    'GENDER',
    'RACE',
    'CITIZENSHIP', 
    'EDU', 
    'INCOME', 
    'HOUSE'
]

prefixes = ['TT', 'TM']
DV_population_top15

,cbsa,GEO_ID,TT_PPL1Y,TT_AGE_LOW,TT_AGE_MID,TT_AGE_HIGH,TT_GENDER_0,TT_GENDER_1,TT_RACE_0,TT_RACE_1,...,TM_INCOME_LOW_LAG_2,SCP_INCOME_LOW_LAG_2,TM_INCOME_MID_LAG_2,SCP_INCOME_MID_LAG_2,TM_INCOME_HIGH_LAG_2,SCP_INCOME_HIGH_LAG_2,TM_HOUSE_0_LAG_2,SCP_HOUSE_0_LAG_2,TM_HOUSE_1_LAG_2,SCP_HOUSE_1_LAG_2
0,35620,6390.0,339.0,19.0,204.0,65.0,187.0,152.0,331.0,8.0,...,80.7,0.380421,0.0,NaN,22.6,0.424779,7.1,1.000000,62.7,0.062201
1,35620,7001.0,18154.0,4218.0,9100.0,1055.0,10865.0,7289.0,9060.0,9094.0,...,20.8,0.442308,25.4,0.677165,5.3,0.754717,1.5,0.400000,13.0,0.661538
2,35620,7002.0,64553.0,14907.0,27596.0,4720.0,31679.0,32874.0,45700.0,18853.0,...,19.0,0.705263,16.5,0.672727,6.3,0.619048,2.2,0.636364,7.5,0.733333
3,35620,7003.0,47069.0,11815.0,20303.0,3301.0,21551.0,25518.0,29372.0,17697.0,...,39.4,0.576142,26.1,0.478927,4.3,0.511628,3.1,0.580645,16.9,0.603550
4,35620,7004.0,7435.0,929.0,3318.0,720.0,3627.0,3808.0,7255.0,180.0,...,20.0,0.720000,6.4,0.000000,0.8,0.000000,0.7,0.000000,18.9,0.724868
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37211,42660,98467.0,14993.0,3136.0,5712.0,1639.0,7305.0,7688.0,8893.0,6100.0,...,66.3,0.787330,37.2,0.604839,14.3,0.657343,6.6,0.833333,24.9,0.694779
37212,42660,98498.0,27765.0,7208.0,10556.0,3379.0,13887.0,13878.0,15849.0,11916.0,...,67.6,0.656805,51.4,0.494163,9.6,0.364583,9.1,0.604396,25.4,0.566929
37213,42660,98499.0,33804.0,10011.0,11601.0,2936.0,16690.0,17114.0,16420.0,17384.0,...,93.9,0.618743,44.1,0.614512,12.8,0.750000,9.9,0.696970,24.9,0.662651
37214,42660,98558.0,60.0,0.0,19.0,12.0,39.0,21.0,60.0,0.0,...,152.9,0.349902,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN


In [9]:
melt_table_final = None

output_path = './Output/final/MERGED_ZCTA/TotalPop/'

for prefix in prefixes:
    for group in groups:
        melt_table = DV_population_top15.melt(id_vars=['year', 'cbsa', 'GEO_ID'], value_name= 'TARGET', var_name='CONTROL', value_vars=[col for col in DV_population_top15.columns if f'{prefix}_{group}' in col and 'LAG' not in col])
        melt_table['CONTROL'] = melt_table['CONTROL'].str.replace(f'{prefix}_', '')
        for lag in range(1, 3):
            melt_table_lag = DV_population_top15.melt(id_vars=['year', 'cbsa', 'GEO_ID'], value_name= f'LAG_{lag}', var_name=f'CONTROL', value_vars=[col for col in DV_population_top15.columns if f'{prefix}_{group}' in col and f'LAG_{lag}' in col])
            melt_table_lag['CONTROL'] = melt_table_lag['CONTROL'].str.replace(f'{prefix}_', '').str.replace(f'_LAG_{lag}', '')
            melt_table = pd.merge(melt_table_lag, melt_table, how='inner', on=['year', 'cbsa', 'GEO_ID', 'CONTROL'])
            
        if melt_table_final is None:
            melt_table_final = melt_table.copy()
        else:
            melt_table_final = pd.merge(melt_table_final, melt_table, how='inner')

        melt_table_final = pd.merge(melt_table_final, all_iv, how='inner', on=['year', 'GEO_ID'])

        melt_table_final['covid'] = (melt_table_final['year'] >= 2020).astype(int)

        melt_table_final.to_csv(f'{output_path}/{prefix}/{prefix}_{group}_TOP15.csv', index=False)
        
        melt_table_final = None

Total Population Change

In [10]:
migration_link = './Output/final/MIGRATION_CHANGE_TOP15.csv.gz'
DV_population_top15 = pd.read_csv(migration_link, compression='gzip')
groups = [
    'PPL1Y', 
    'AGE',
    'GENDER',
    'RACE',
    'CITIZENSHIP', 
    'EDU', 
    'INCOME', 
    'HOUSE'
]

prefixes = ['TT', 'TM']
DV_population_top15

,GEO_ID,year,cbsa,TT_PPL1Y,TT_AGE_LOW,TT_AGE_MID,TT_AGE_HIGH,TT_GENDER_0,TT_GENDER_1,TT_RACE_0,...,TM_CITIZENSHIP_0_LAG_2,TM_CITIZENSHIP_1_LAG_2,TM_EDU_LOW_LAG_2,TM_EDU_MID_LAG_2,TM_EDU_HIGH_LAG_2,TM_INCOME_LOW_LAG_2,TM_INCOME_MID_LAG_2,TM_INCOME_HIGH_LAG_2,TM_HOUSE_0_LAG_2,TM_HOUSE_1_LAG_2
0,6390.0,2016,35620,-36.0,0.0,-43.0,-1.0,-16.0,-20.0,-34.0,...,-7.6,0.0,0.0,-31.3,-4.8,-14.0,0.0,-10.7,-7.1,6.2
1,7001.0,2016,35620,-178.0,-145.0,115.0,77.0,43.0,-221.0,-202.0,...,-0.6,-0.4,1.5,-4.0,-0.3,4.2,-7.3,0.5,1.7,-3.5
2,7002.0,2016,35620,458.0,435.0,-724.0,386.0,840.0,-382.0,-1827.0,...,1.4,1.4,0.7,2.0,3.0,1.5,1.1,1.0,-0.1,0.6
3,7003.0,2016,35620,179.0,-364.0,84.0,350.0,373.0,-194.0,446.0,...,-1.6,-1.5,-2.5,1.4,0.1,-1.3,2.8,0.2,0.6,-1.2
4,7004.0,2016,35620,-126.0,173.0,-172.0,5.0,-161.0,35.0,-253.0,...,8.5,-0.9,-2.0,5.5,2.8,1.0,13.4,0.1,0.8,2.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32559,98467.0,2022,42660,-196.0,-269.0,146.0,89.0,-21.0,-175.0,-550.0,...,-8.8,-6.0,-3.2,-3.5,-1.7,-20.9,-23.3,3.6,-1.9,-4.6
32560,98498.0,2022,42660,-722.0,-393.0,208.0,29.0,143.0,-865.0,-1760.0,...,13.4,5.1,-0.2,3.8,1.8,-2.0,1.5,1.8,0.5,-1.7
32561,98499.0,2022,42660,726.0,378.0,-129.0,97.0,369.0,357.0,-115.0,...,-3.2,0.8,2.3,-3.3,-0.8,0.4,-11.1,-0.3,-2.7,-2.3
32562,98558.0,2022,42660,9.0,0.0,1.0,0.0,1.0,8.0,20.0,...,31.2,0.0,0.0,1.9,0.0,6.7,0.0,0.0,0.0,0.0


In [11]:
# create dataset for total population changes and total migration flow changes subject to multiple control groups
melt_table_final = None

output_path = './Output/final/MERGED_ZCTA/TotalPopChange/'

for prefix in prefixes:
    for group in groups:
        melt_table = DV_population_top15.melt(id_vars=['year', 'cbsa', 'GEO_ID'], value_name= 'TARGET', var_name='CONTROL', value_vars=[col for col in DV_population_top15.columns if f'{prefix}_{group}' in col and 'LAG' not in col])
        melt_table['CONTROL'] = melt_table['CONTROL'].str.replace(f'{prefix}_', '')
        for lag in range(1, 3):
            melt_table_lag = DV_population_top15.melt(id_vars=['year', 'cbsa', 'GEO_ID'], value_name= f'LAG_{lag}', var_name=f'CONTROL', value_vars=[col for col in DV_population_top15.columns if f'{prefix}_{group}' in col and f'LAG_{lag}' in col])
            melt_table_lag['CONTROL'] = melt_table_lag['CONTROL'].str.replace(f'{prefix}_', '').str.replace(f'_LAG_{lag}', '')
            melt_table = pd.merge(melt_table_lag, melt_table, how='inner', on=['year', 'cbsa', 'GEO_ID', 'CONTROL'])
            
        if melt_table_final is None:
            melt_table_final = melt_table.copy()
        else:
            melt_table_final = pd.merge(melt_table_final, melt_table, how='inner')

        melt_table_final = pd.merge(melt_table_final, all_iv, how='inner', on=['year', 'GEO_ID'])

        melt_table_final['covid'] = (melt_table_final['year'] >= 2020).astype(int)

        melt_table_final.to_csv(f'{output_path}/{prefix}/{prefix}_{group}_TOP15.csv', index=False)
        
        melt_table_final = None

In [14]:
migration_link = './Output/final/MIGRATION_CBD.csv.gz'
DV_population_cbd = pd.read_csv(migration_link, compression='gzip')

output_path = './Output/final/MERGED_ZCTA/'

for group in groups:
    melt_table = DV_population_cbd.melt(id_vars=['year', 'cbsa', 'GEO_ID'], value_name= 'TARGET', var_name='CONTROL', value_vars=[col for col in DV_population_cbd.columns if f'SCP_{group}' in col and 'LAG' not in col])
    melt_table['CONTROL'] = melt_table['CONTROL'].str.replace(f'{prefix}_', '')
    for lag in range(1, 3):
        melt_table_lag = DV_population_cbd.melt(id_vars=['year', 'cbsa', 'GEO_ID'], value_name= f'LAG_{lag}', var_name=f'CONTROL', value_vars=[col for col in DV_population_cbd.columns if f'SCP_{group}' in col and f'LAG_{lag}' in col])
        melt_table_lag['CONTROL'] = melt_table_lag['CONTROL'].str.replace(f'{prefix}_', '').str.replace(f'_LAG_{lag}', '')
        melt_table = pd.merge(melt_table_lag, melt_table, how='inner', on=['year', 'cbsa', 'GEO_ID', 'CONTROL'])
        
    if melt_table_final is None:
        melt_table_final = melt_table.copy()
    else:
        melt_table_final = pd.merge(melt_table_final, melt_table, how='inner')

    melt_table_final = pd.merge(melt_table_final, all_iv, how='inner', on=['year', 'GEO_ID'])

    melt_table_final['covid'] = (melt_table_final['year'] >= 2020).astype(int)

    melt_table_final.to_csv(f'{output_path}/SCP/SCP_{group}_CBD.csv', index=False)
    
    melt_table_final = None

In [3]:
! gzip ./Output/final/MERGED_ZCTA ./Output/final/MERGED_ZCTA.gz

'gzip' is not recognized as an internal or external command,
operable program or batch file.
